# tuning

In [1]:
import torch
import torch.nn as nn
import torchvision
import optuna
import adabound

In [2]:
import tqdm
import yaml

from addict import Dict
from tensorboardX import SummaryWriter
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, RandomCrop, Normalize

from utils.class_weight import get_class_weight
from utils.dataset import Kinetics
from utils.mean import get_mean, get_std

from model.resnet import resnet18

In [3]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()

    epoch_loss = 0.0
    for i, sample in tqdm.tqdm(enumerate(train_loader), total=100):
        x = sample['clip']
        t = sample['cls_id']
        x = x.to(device)
        t = t.to(device)

        h = model(x)
        loss = criterion(h, t)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        
        if i == 100:
            break

    return epoch_loss / i


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions"""
    N = output.shape[0]
    maxk = max(topk)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
        res.append(correct_k)
    return N, res


def validation(model, val_loader, criterion, config, device):
    model.eval()
    val_loss = 0.0
    n_samples = 0.0
    top1 = 0.0
    top5 = 0.0

    with torch.no_grad():
        for i, sample in enumerate(val_loader):
            # temporal size is input_frames(default 16) * interger
            x = sample['clip']
            x = x.to(device)
            t = sample['cls_id']
            t = t.to(device)

            h = model(x)
            val_loss += criterion(h, t).item()
            n, topk = accuracy(h, t, topk=(1, 5))
            n_samples += n
            top1 += topk[0].item()
            top5 += topk[1].item()
            
            if i==100:
                break

        val_loss /= i
        top1 /= n_samples
        top5 /= n_samples

    return val_loss, top1, top5

In [4]:
def get_optimizer(trial, model):
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-17, 1)
    lr = trial.suggest_loguniform('lr', 1e-7, 1e-1)
    optimizer = adabound.AdaBound(model.parameters(), lr=lr, weight_decay=weight_decay)
    return optimizer

In [5]:
# configuration
CONFIG = Dict(yaml.safe_load(open('./result/resnet18/adabound/config.yaml')))

# DataLoaders
normalize = Normalize(mean=get_mean(), std=get_std())

train_data = Kinetics(
    CONFIG,
    transform=Compose([
        RandomCrop((CONFIG.height, CONFIG.width)),
        ToTensor(),
        normalize,
    ])
)

val_data = Kinetics(
    CONFIG,
    transform=Compose([
        RandomCrop((CONFIG.height, CONFIG.width)),
        ToTensor(),
        normalize,
    ]),
    mode='validation'
)

train_loader = DataLoader(
    train_data,
    batch_size=16,
    shuffle=False,
    num_workers=1,
    drop_last=True
)

val_loader = DataLoader(
    val_data,
    batch_size=16,
    shuffle=False,
    num_workers=1
)

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1, 2"

In [7]:
device = 'cuda'
criterion = nn.criterion = nn.CrossEntropyLoss(
            weight=get_class_weight().to(device))

In [8]:
def objective(trial):
    model = resnet18(num_classes=CONFIG.n_classes)
    model = nn.DataParallel(model)
    model.to(device)
    optimizer = get_optimizer(trial, model)

    for step in range(1):
        _ = train(model, train_loader, criterion, optimizer, device)
        _, cor, _ = validation(model, val_loader, criterion, CONFIG, device)
        error_rate = 1 - cor
    return error_rate

study = optuna.create_study()
study.optimize(objective, n_trials=20)

/home/cvrg/Documents/ActionRecognition/model/resnet.py:146: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
100%|██████████| 100/100 [01:34<00:00,  1.11it/s][I 2019-05-08 12:03:21,199] Finished trial#0 resulted in value: 0.9399752475247525. Current best value is 0.9399752475247525 with parameters: {'weight_decay': 4.961632044922973e-17, 'lr': 0.001208774195160568}.

100%|██████████| 100/100 [01:28<00:00,  1.16it/s][I 2019-05-08 12:06:17,761] Finished trial#1 resulted in value: 0.9393564356435644. Current best value is 0.9393564356435644 with parameters: {'weight_decay': 3.20444277196927e-17, 'lr': 0.00020025085875225815}.


  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:01<02:14,  1.36s/it]

  2%|▏         | 2/100 [00:02<01:57,  1.20s/it]

  3%|▎         | 3/100 [00:03<01:48,  1.12s/it]

  4%|▍         | 4/100 [00:03<01:39,  1.04s/it]

  5%|▌         | 5/100 [00

 51%|█████     | 51/100 [00:45<00:43,  1.12it/s]

 52%|█████▏    | 52/100 [00:46<00:42,  1.14it/s]

 53%|█████▎    | 53/100 [00:47<00:41,  1.13it/s]

 54%|█████▍    | 54/100 [00:48<00:39,  1.16it/s]

 55%|█████▌    | 55/100 [00:48<00:38,  1.15it/s]

 56%|█████▌    | 56/100 [00:49<00:37,  1.16it/s]

 57%|█████▋    | 57/100 [00:50<00:37,  1.15it/s]

 58%|█████▊    | 58/100 [00:51<00:35,  1.18it/s]

 59%|█████▉    | 59/100 [00:52<00:35,  1.17it/s]

 60%|██████    | 60/100 [00:53<00:34,  1.17it/s]

 61%|██████    | 61/100 [00:54<00:33,  1.15it/s]

 62%|██████▏   | 62/100 [00:54<00:32,  1.16it/s]

 63%|██████▎   | 63/100 [00:55<00:32,  1.15it/s]

 64%|██████▍   | 64/100 [00:56<00:30,  1.16it/s]

 65%|██████▌   | 65/100 [00:57<00:30,  1.15it/s]

 66%|██████▌   | 66/100 [00:58<00:29,  1.16it/s]

 67%|██████▋   | 67/100 [00:59<00:28,  1.15it/s]

 68%|██████▊   | 68/100 [01:00<00:27,  1.16it/s]

 69%|██████▉   | 69/100 [01:01<00:27,  1.14it/s]

 70%|███████   | 70/100 [01:01<00:25,  1.17it/s]



  2%|▏         | 2/100 [00:02<01:58,  1.21s/it]



  3%|▎         | 3/100 [00:03<01:48,  1.12s/it]



  4%|▍         | 4/100 [00:03<01:40,  1.04s/it]



  5%|▌         | 5/100 [00:04<01:34,  1.00it/s]



  6%|▌         | 6/100 [00:05<01:31,  1.03it/s]



  7%|▋         | 7/100 [00:06<01:28,  1.05it/s]



  8%|▊         | 8/100 [00:07<01:24,  1.09it/s]



  9%|▉         | 9/100 [00:08<01:23,  1.09it/s]



 10%|█         | 10/100 [00:09<01:21,  1.10it/s]



 11%|█         | 11/100 [00:10<01:20,  1.11it/s]



 12%|█▏        | 12/100 [00:11<01:16,  1.14it/s]



 13%|█▎        | 13/100 [00:11<01:16,  1.13it/s]



 14%|█▍        | 14/100 [00:12<01:15,  1.14it/s]



 15%|█▌        | 15/100 [00:13<01:15,  1.13it/s]



 16%|█▌        | 16/100 [00:14<01:11,  1.17it/s]



 17%|█▋        | 17/100 [00:15<01:12,  1.14it/s]



 18%|█▊        | 18/100 [00:16<01:10,  1.16it/s]



 19%|█▉        | 19/100 [00:17<01:10,  1.15it/s]



 20%|██        | 20/100 [00:17<01:08,  1.16it/s]



 21%|██        | 21/

 53%|█████▎    | 53/100 [00:47<00:41,  1.14it/s]




 54%|█████▍    | 54/100 [00:47<00:39,  1.16it/s]




 55%|█████▌    | 55/100 [00:48<00:39,  1.15it/s]




 56%|█████▌    | 56/100 [00:49<00:37,  1.17it/s]




 57%|█████▋    | 57/100 [00:50<00:37,  1.16it/s]




 58%|█████▊    | 58/100 [00:51<00:35,  1.18it/s]




 59%|█████▉    | 59/100 [00:52<00:35,  1.17it/s]




 60%|██████    | 60/100 [00:53<00:33,  1.18it/s]




 61%|██████    | 61/100 [00:53<00:33,  1.17it/s]




 62%|██████▏   | 62/100 [00:54<00:32,  1.18it/s]




 63%|██████▎   | 63/100 [00:55<00:31,  1.16it/s]




 64%|██████▍   | 64/100 [00:56<00:30,  1.17it/s]




 65%|██████▌   | 65/100 [00:57<00:30,  1.17it/s]




 66%|██████▌   | 66/100 [00:58<00:29,  1.17it/s]




 67%|██████▋   | 67/100 [00:59<00:28,  1.15it/s]




 68%|██████▊   | 68/100 [00:59<00:27,  1.17it/s]




 69%|██████▉   | 69/100 [01:00<00:26,  1.16it/s]




 70%|███████   | 70/100 [01:01<00:25,  1.17it/s]




 71%|███████   | 71/100 [01:02<00:25,  1.16it/

100%|██████████| 100/100 [01:25<00:00,  1.20it/s][I 2019-05-08 12:21:00,136] Finished trial#6 resulted in value: 0.9721534653465347. Current best value is 0.8997524752475248 with parameters: {'weight_decay': 4.027614480048272e-05, 'lr': 1.0853536931549944e-05}.







  0%|          | 0/100 [00:00<?, ?it/s]






  1%|          | 1/100 [00:01<02:09,  1.31s/it]






  2%|▏         | 2/100 [00:02<01:55,  1.17s/it]






  3%|▎         | 3/100 [00:03<01:45,  1.09s/it]






  4%|▍         | 4/100 [00:03<01:38,  1.02s/it]






  5%|▌         | 5/100 [00:04<01:33,  1.01it/s]






  6%|▌         | 6/100 [00:05<01:27,  1.07it/s]






  7%|▋         | 7/100 [00:06<01:26,  1.08it/s]






  8%|▊         | 8/100 [00:07<01:23,  1.11it/s]






  9%|▉         | 9/100 [00:08<01:23,  1.09it/s]






 10%|█         | 10/100 [00:09<01:21,  1.11it/s]






 11%|█         | 11/100 [00:10<01:19,  1.12it/s]






 12%|█▏        | 12/100 [00:10<01:17,  1.14it/s]






 13%|█▎        | 13/100 [00:11<01:

...

100%|██████████| 100/100 [01:26<00:00,  1.18it/s][I 2019-05-08 12:44:29,107] Finished trial#14 resulted in value: 0.9467821782178217. Current best value is 0.8997524752475248 with parameters: {'weight_decay': 4.027614480048272e-05, 'lr': 1.0853536931549944e-05}.















  0%|          | 0/100 [00:00<?, ?it/s]














  1%|          | 1/100 [00:01<02:12,  1.34s/it]














  2%|▏         | 2/100 [00:02<01:57,  1.20s/it]














  3%|▎         | 3/100 [00:03<01:47,  1.11s/it]














  4%|▍         | 4/100 [00:03<01:37,  1.02s/it]














  5%|▌         | 5/100 [00:04<01:34,  1.01it/s]














  6%|▌         | 6/100 [00:05<01:29,  1.05it/s]














  7%|▋         | 7/100 [00:06<01:26,  1.08it/s]














  8%|▊         | 8/100 [00:07<01:23,  1.10it/s]














  9%|▉         | 9/100 [00:08<01:23,  1.10it/s]














 10%|█         | 10/100 [00:09<01:20,  1.11it/s]














 11%|█         | 11/100 [00:10<01:19,  1.12it/s

 21%|██        | 21/100 [00:18<01:09,  1.14it/s]















 22%|██▏       | 22/100 [00:19<01:07,  1.15it/s]















 23%|██▎       | 23/100 [00:20<01:08,  1.12it/s]















 24%|██▍       | 24/100 [00:21<01:06,  1.15it/s]















 25%|██▌       | 25/100 [00:22<01:06,  1.12it/s]















 26%|██▌       | 26/100 [00:23<01:05,  1.14it/s]















 27%|██▋       | 27/100 [00:24<01:04,  1.13it/s]















 28%|██▊       | 28/100 [00:24<01:02,  1.15it/s]















 29%|██▉       | 29/100 [00:25<01:02,  1.13it/s]















 30%|███       | 30/100 [00:26<01:01,  1.14it/s]















 31%|███       | 31/100 [00:27<01:01,  1.12it/s]















 32%|███▏      | 32/100 [00:28<01:00,  1.12it/s]















 33%|███▎      | 33/100 [00:29<01:00,  1.10it/s]















 34%|███▍      | 34/100 [00:30<00:58,  1.12it/s]















 35%|███▌      | 35/100 [00:31<00:58,  1.11it/s]















 36%|███▌      | 36/100 [00:32<00:56,  1

 44%|████▍     | 44/100 [00:39<00:50,  1.11it/s]
















 45%|████▌     | 45/100 [00:40<00:49,  1.10it/s]
















 46%|████▌     | 46/100 [00:40<00:48,  1.12it/s]
















 47%|████▋     | 47/100 [00:41<00:47,  1.10it/s]
















 48%|████▊     | 48/100 [00:42<00:46,  1.12it/s]
















 49%|████▉     | 49/100 [00:43<00:46,  1.11it/s]
















 50%|█████     | 50/100 [00:44<00:44,  1.12it/s]
















 51%|█████     | 51/100 [00:45<00:43,  1.12it/s]
















 52%|█████▏    | 52/100 [00:46<00:42,  1.14it/s]
















 53%|█████▎    | 53/100 [00:47<00:41,  1.15it/s]
















 54%|█████▍    | 54/100 [00:47<00:39,  1.16it/s]
















 55%|█████▌    | 55/100 [00:48<00:39,  1.15it/s]
















 56%|█████▌    | 56/100 [00:49<00:37,  1.16it/s]
















 57%|█████▋    | 57/100 [00:50<00:36,  1.16it/s]
















 58%|█████▊    | 58/100 [00:51<00:35,  1.18it/s]
















 59%|█████▉    | 59/100 [

 65%|██████▌   | 65/100 [00:57<00:30,  1.15it/s]

















 66%|██████▌   | 66/100 [00:58<00:29,  1.16it/s]

















 67%|██████▋   | 67/100 [00:59<00:28,  1.15it/s]

















 68%|██████▊   | 68/100 [01:00<00:27,  1.16it/s]

















 69%|██████▉   | 69/100 [01:01<00:27,  1.14it/s]

















 70%|███████   | 70/100 [01:01<00:25,  1.17it/s]

















 71%|███████   | 71/100 [01:02<00:25,  1.16it/s]

















 72%|███████▏  | 72/100 [01:03<00:24,  1.16it/s]

















 73%|███████▎  | 73/100 [01:04<00:23,  1.15it/s]

















 74%|███████▍  | 74/100 [01:05<00:22,  1.17it/s]

















 75%|███████▌  | 75/100 [01:06<00:21,  1.16it/s]

















 76%|███████▌  | 76/100 [01:07<00:20,  1.17it/s]

















 77%|███████▋  | 77/100 [01:07<00:20,  1.14it/s]

















 78%|███████▊  | 78/100 [01:08<00:18,  1.18it/s]

















 79%|███████▉  | 79/100 [01:09<00:18,  1.17it/s]

















 80%|█████

 84%|████████▍ | 84/100 [01:13<00:13,  1.20it/s]


















 85%|████████▌ | 85/100 [01:14<00:12,  1.18it/s]


















 86%|████████▌ | 86/100 [01:15<00:11,  1.20it/s]


















 87%|████████▋ | 87/100 [01:16<00:10,  1.19it/s]


















 88%|████████▊ | 88/100 [01:17<00:09,  1.20it/s]


















 89%|████████▉ | 89/100 [01:18<00:09,  1.17it/s]


















 90%|█████████ | 90/100 [01:18<00:08,  1.19it/s]


















 91%|█████████ | 91/100 [01:19<00:07,  1.18it/s]


















 92%|█████████▏| 92/100 [01:20<00:06,  1.18it/s]


















 93%|█████████▎| 93/100 [01:21<00:05,  1.17it/s]


















 94%|█████████▍| 94/100 [01:22<00:05,  1.18it/s]


















 95%|█████████▌| 95/100 [01:23<00:04,  1.18it/s]


















 96%|█████████▌| 96/100 [01:23<00:03,  1.20it/s]


















 97%|█████████▋| 97/100 [01:24<00:02,  1.18it/s]


















 98%|█████████▊| 98/100 [01:25<00:01,  1.20it/s]















In [9]:
study.best_params

{'weight_decay': 4.027614480048272e-05, 'lr': 1.0853536931549944e-05}

In [10]:
study.best_value

0.8997524752475248

In [11]:
df = study.trials_dataframe() 

In [12]:
df.head()

number                state     value             datetime_start  \
                                                                     
0      0  TrialState.COMPLETE  0.939975 2019-05-08 12:00:09.059177   
1      1  TrialState.COMPLETE  0.939356 2019-05-08 12:03:21.202255   
2      2  TrialState.COMPLETE  0.904703 2019-05-08 12:06:17.764285   
3      3  TrialState.COMPLETE  0.938738 2019-05-08 12:09:15.134594   
4      4  TrialState.COMPLETE  0.899752 2019-05-08 12:12:12.660499   

           datetime_complete    params               system_attrs  
                                    lr  weight_decay      _number  
0 2019-05-08 12:03:21.198913  0.001209  4.961632e-17            0  
1 2019-05-08 12:06:17.760572  0.000200  3.204443e-17            1  
2 2019-05-08 12:09:15.129254  0.000010  8.835384e-16            2  
3 2019-05-08 12:12:12.656506  0.000037  1.182161e-02            3  
4 2019-05-08 12:15:10.275040  0.000011  4.027614e-05            4